# Load Text

서로 다른 작가가 호머의 일리아드를 영어로 번역한 3개의 텍스트를 읽어 들여 적절하게 데이터 전처리를 수행하고 데이터 셋으로 생성한다.

In [1]:
import tensorflow as tf
print(tf.__version__)

#import tensorflow_datasets as tfds
import os

2.11.0


### Text 파일 준비하기

호머의 일리아드를 영어로 번역한 3개의 텍스트를 사용한다.

In [2]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
#DIRECTORY_URL = os.environ['USERPROFILE'] + '\\.keras\\datasets'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

'C:\\Users\\USER\\.keras\\datasets'

3개의 텍스트 파일을 읽어들여 하나의 데이터셋으로 만든다.

In [3]:
data = []     # 텍스트
labels = []   # labels 0=cowper, 1=derby, 2=butler
for i, file_name in enumerate(FILE_NAMES):
    with open(os.path.join(parent_dir, file_name)) as f:
        lines = f.readlines()
        cls = [i] * len(lines)
        print([i] , len(lines))
        data.extend(lines) 
        labels.extend(cls)

data[0:3], labels[0:3]
data[19143:19146], labels[19143:19146]

[0] 19143
[1] 18334
[2] 12131


(["癤풭f Peleus' son, Achilles, sing, O Muse,\n",
  'The vengeance, deep and deadly; whence to Greece\n',
  'Unnumbered ills arose; which many a soul\n'],
 [1, 1, 1])

### Tokenizer

* Tokenizer API를 사용하여 해당 문서를 token으로 분리한다.
* texts_to_sequences() 로 해당 단어를 숫자로 encode한다.
* pad_sequences()를 사용하면 해당 단어의 sequence의 길이를 모두 동일하게 맞출 수 있다.

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer

def tokenize(lang):
    lang_tokenizer = Tokenizer()
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

In [35]:
tensor, tokenizer = tokenize(data)

tensor[0], tokenizer.index_word

(array([10414,  5198,   381,   257,   429,    21,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]),
 {1: 'the',
  2: 'and',
  3: 'of',
  4: 'to',
  5: 'his',
  6: 'he',
  7: 'in',
  8: 'with',
  9: 'a',
  10: 'him',
  11: 'from',
  12: 'for',
  13: 'but',
  14: 'i',
  15: 'on',
  16: 'that',
  17: 'all',
  18: 'as',
  19: 'by',
  20: 'they',
  21: 'son',
  22: 'their',
  23: 'not',
  24: 'my',
  25: 'then',
  26: 'at',
  27: 'it',
  28: 'was',
  29: 'you',
  30: 'so',
  31: 'them',
  32: 'me',
  33: 'who',
  34: 'had',
  35: 'when',
  36: 'thus',
  37: 'is',
  38: 'thou',
  39: 'now',
  40: 'her',
  41: 'will',
  42: 'be',
  43: 'hector',
  44: 'thy',
  45: 'or',
  46: 'jove',
  47: 'have',
  48: 'shall',
  49: 'were',
  50: 'achilles',
  51: 'trojans',
  52: 'spear',
  53: 'nor',
  54: 'whom',
  55: 'we',
  56: 'this',
  57: 'ships',
  58: 'she',
  59: 'if',
  60: 'are',
  61: 'hand',
  62: 'which',
  63: 'there',
  64: 'no',


`index_word()`  
tokenize 결과를 다시 단어 시퀀스로 변환하는 것도 가능하다.

In [6]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [7]:
convert(tokenizer, tensor[0])

10414 ----> 癤풞chilles
5198 ----> sing
381 ----> o
257 ----> goddess
429 ----> peleus'
21 ----> son


##  tf.data 데이터 세트 만들기

딥러닝 모형에 입력으로 사용할 수 있게 텍스트 데이터를 데이터셋으로 만든다.

In [8]:
BUFFER_SIZE = len(tensor)
BATCH_SIZE = 64
vocab_size = len(tokenizer.word_index)

print(vocab_size)

15537


In [9]:
dataset = tf.data.Dataset.from_tensor_slices((tensor, labels)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [10]:
next(iter(dataset))

(<tf.Tensor: shape=(64, 19), dtype=int32, numpy=
 array([[7166,    2,  440, ...,    0,    0,    0],
        [   1, 5016, 3257, ...,    0,    0,    0],
        [2309,   12,  186, ...,    0,    0,    0],
        ...,
        [   1,  484,  285, ...,    0,    0,    0],
        [ 831,    4,  965, ...,    0,    0,    0],
        [   6,  109,    1, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(64,), dtype=int32, numpy=
 array([0, 1, 0, 2, 0, 2, 0, 0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 1, 2, 2, 2, 0,
        2, 0, 0, 1, 0, 2, 2, 1, 2, 0, 2, 1, 1, 1, 0, 0, 2, 2, 0, 1, 2, 1,
        2, 1, 2, 0, 1, 2, 1, 2, 0, 1, 1, 0, 2, 0, 0, 0, 1, 0, 1, 1])>)